In [9]:
pip install tpot

Note: you may need to restart the kernel to use updated packages.Collecting tpot
  Obtaining dependency information for tpot from https://files.pythonhosted.org/packages/c0/9d/a5a5422e00c034e16c8ad2e0361cf26578a8fcbbc45c6d605d2decf72bc7/TPOT-0.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn>=1.4.1 from https://files.pythonhosted.org/packages/79/3d/02d5d3ed359498fec3abdf65407d3c07e3b8765af17464969055aaec5171/scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata
  Using cached deap-1.4.1-py3-none-any.whl
  Obtaining dependency information for update-checker>=0.16 from https://files.pythonhosted.org/packages/0c/ba/8dd7fa5f0b1c6a8ac62f8f57f7e794160c1f86f31c6d0fb00f582372a3e4/update_checker-0.18.0-py3-none-any.whl.metadata
  Using cached stopit-1.1.2-py3-none-any.whl
  Obtaining dependency information for xgboost>=1.1.0 from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [10]:
# Importo las librerias
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import numpy as np

C:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [11]:
# Cargo el dataset como un dataframe
df = pd.read_csv('predictive_maintenance.csv')

In [12]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [13]:
df.tail()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure
9999,10000,M24859,M,299.0,308.7,1500,40.2,30,0,No Failure


In [14]:
df.dtypes

UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Target                       int64
Failure Type                object
dtype: object

In [16]:
# TPot no tiene ningun preprocesamiento cuando la columna es de tipo string. Se tiene que realizar el preprocesamiento de forma manual
columnasObject = df.select_dtypes(include=object).columns
for columna in columnasObject:
  df = pd.concat([df.drop(columna, axis=1), pd.get_dummies(df[columna])], axis=1)

In [17]:
df.head()

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H29424,H29425,H29432,...,M24859,H,L,M,Heat Dissipation Failure,No Failure,Overstrain Failure,Power Failure,Random Failures,Tool Wear Failure
0,1,298.1,308.6,1551,42.8,0,0,False,False,False,...,False,False,False,True,False,True,False,False,False,False
1,2,298.2,308.7,1408,46.3,3,0,False,False,False,...,False,False,True,False,False,True,False,False,False,False
2,3,298.1,308.5,1498,49.4,5,0,False,False,False,...,False,False,True,False,False,True,False,False,False,False
3,4,298.2,308.6,1433,39.5,7,0,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4,5,298.2,308.7,1408,40.0,9,0,False,False,False,...,False,False,True,False,False,True,False,False,False,False


In [18]:
df.tail()

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H29424,H29425,H29432,...,M24859,H,L,M,Heat Dissipation Failure,No Failure,Overstrain Failure,Power Failure,Random Failures,Tool Wear Failure
9995,9996,298.8,308.4,1604,29.5,14,0,False,False,False,...,False,False,False,True,False,True,False,False,False,False
9996,9997,298.9,308.4,1632,31.8,17,0,False,False,False,...,False,True,False,False,False,True,False,False,False,False
9997,9998,299.0,308.6,1645,33.4,22,0,False,False,False,...,False,False,False,True,False,True,False,False,False,False
9998,9999,299.0,308.7,1408,48.5,25,0,False,False,False,...,False,True,False,False,False,True,False,False,False,False
9999,10000,299.0,308.7,1500,40.2,30,0,False,False,False,...,True,False,False,True,False,True,False,False,False,False


In [21]:
# Separo el modleo entre train y test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Target'], axis=1), df.Target, train_size=0.75, test_size=0.25, random_state=42)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500, 10015)
(2500, 10015)
(7500,)
(2500,)


In [23]:
# Configuracion Default
generations=2
population_size=5
pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1')

In [24]:
pipeline_optimizer.fit(X_train, y_train)

                                                                                                                       
Generation 1 - Current best internal CV score: 0.9791534574709233
                                                                                                                       
Generation 2 - Current best internal CV score: 0.9865557096625057
                                                          
Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=5, min_samples_leaf=6, min_samples_split=4)


TPOTClassifier(generations=2, memory='auto', n_jobs=-1, population_size=5,
               random_state=42, scoring='f1', verbosity=2)

In [25]:
print(pipeline_optimizer.score(X_test, y_test))

0.9859154929577465


In [26]:
pipeline_optimizer.export('tpot_export_default_pipeline.py')

In [27]:
#Configuracion Light
pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1',
    config_dict='TPOT light')

In [28]:
pipeline_optimizer.fit(X_train, y_train)

                                                                                                                       
Generation 1 - Current best internal CV score: 0.9646536164490218
                                                                                                                       
Generation 2 - Current best internal CV score: 0.9712610894116935
                                                          
Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=9, min_samples_leaf=12, min_samples_split=7)


TPOTClassifier(config_dict='TPOT light', generations=2, memory='auto',
               n_jobs=-1, population_size=5, random_state=42, scoring='f1',
               verbosity=2)

In [29]:
print(pipeline_optimizer.score(X_test, y_test))

0.9859154929577465


In [30]:
pipeline_optimizer.export('tpot_export_light_pipeline.py')

In [31]:
# Configuracion Custom
config_dict = {
     'sklearn.ensemble.RandomForestClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf':  range(1, 21),
        'bootstrap': [True, False]
    }
}

pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1',
    config_dict=config_dict)

In [32]:
pipeline_optimizer.fit(X_train, y_train)

                                                                                                                       
Generation 1 - Current best internal CV score: 0.9828520242762864
                                                           
Generation 2 - Current best internal CV score: 0.9828520242762864
                                                           
Best pipeline: RandomForestClassifier(RandomForestClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.15000000000000002, min_samples_leaf=18, min_samples_split=14, n_estimators=100), bootstrap=False, criterion=entropy, max_features=0.35000000000000003, min_samples_leaf=18, min_samples_split=14, n_estimators=100)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'bootstrap': [True,
                                                                                      False],
                                                                        'criterion': ['gini',
                                                                                      'entropy'],
                                                                        'max_features': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                                        'min_samples_leaf': range(1, 21),
                                                                        'min_samples_split': range(2, 21),
                                                                        'n_estimators': [100]}},
               generations=2, memory='auto', n_jobs=-1, population_size=5,
               random_state=42, scoring='f1', verbosity=2)

In [33]:
print(pipeline_optimizer.score(X_test, y_test))

C:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.9859154929577465


In [34]:
pipeline_optimizer.export('tpot_export_custom_pipeline.py')